# Tugas Besar 1 Machine Learning

## About
Feedforward Neural Network algorithm with rectified linear unit (ReLU) and sigmoid activation function.

### Features

### Usage

### Tech stack
* ![Python](https://img.shields.io/badge/Python-3776AB?style=for-the-badge&logo=python&logoColor=white)

## Author
| Name | NIM |
| :--- | :---: |
| Angger Ilham Amanullah | 13521001 |
| Kelvin Rayhan Alkarim | 13521005 |
| Ditra Rizqa Amadia | 13521019 |
| Bernardus Willson | 13521021 |

<hr>

> Make sure required modules are imported

In [271]:
# Modules
import json
import numpy as np
from graphviz import Digraph, Source
from IPython.display import display
import pickle
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

### Classes

In [272]:
class Driver:
    @staticmethod
    def terminate(code):
        print(f"[■] Program terminated with code {code}")
        quit(code)

    @staticmethod
    def load_model(model_file):
        try:
            model = json.load(open(model_file, "r"))
        except:
            print(f"[✖] Error loading model from '{model_file}'")
            print("")

            Driver.terminate(1)        
            
        print(f"[✔] Successfully loaded model from '{model_file}'")
        print("")

        return model

In [273]:
class Layer:
    # Initialize layer
    def __init__(self, id, layer, weight, input):
        try:
            self._id = id
            self._num_of_neuron = layer["number_of_neurons"]
            self._activation_function = layer["activation_function"]
            self._weight = weight
            self._input = input       
            self._input = np.insert(self._input, 0, 1) # Append bias of 1
            self._sigma = np.zeros(self._num_of_neuron)
            self._output = np.zeros(self._num_of_neuron)
            self._error = np.zeros(self._num_of_neuron)
            self._error_node_output = np.zeros(self._num_of_neuron)
            self._loss = 0
        except:
            print(f"[✖] Error initialiazing layer")
            print("")

            Driver.terminate(1)

    # Calculate sigma
    def _calculate_sigma(self, i):
        for j in range(len(self._input)):
            self._sigma[i] += self._input[j] * self._weight[j][i]

    # Calculate activation
    def _calculate_activation(self, i):
        if self._activation_function == 'relu':
            self._output[i] = max(0, self._sigma[i])
        elif self._activation_function == 'sigmoid':
            self._output[i] = 1 / (1 + np.exp(-self._sigma[i]))
        elif self._activation_function == 'linear':
            self._output[i] = self._sigma[i]
        elif self._activation_function == 'softmax':
            exp_values = np.exp(self._sigma - np.max(self._sigma))
            self._output = exp_values / np.sum(exp_values)

    # Calculate error
    def _calculate_error(self, target, i, isOutput):
        if isOutput:
            self._error[i] = target[i] - self._output[i]
        else:
            for j in range(len(target)):
                self._error[i] += target[j] * self._weight[i+1][j]

    # Calculate ENO
    def _calculate_derivative(self, i):
        if self._activation_function == 'relu':
            self._error_node_output[i] = self._error[i] if self._sigma[i] > 0 else 0
        elif self._activation_function == 'sigmoid':
            self._error_node_output[i] = self._error[i] * self._sigma[i] * (1 - self._sigma[i])
        elif self._activation_function == 'linear':
            self._error_node_output[i] = self._error[i]
        elif self._activation_function == 'softmax':
            pass

    # Calculate forward
    def _calculate_forward(self):
        print(f"[●] Building layer {self._id}")

        # For each neuron, calculate its activation
        for i in range(self._num_of_neuron):
            self._calculate_sigma(i)
            self._calculate_activation(i)

        print(f"[✔] Successfully built layer {self._id}")
        print(f"Number of neurons: {self._num_of_neuron}")
        print(f"Activation function: {self._activation_function}")
        for i in range(len(self._output)):
            print(f"h{self._id}{i}: {self._output[i]}")
        print("")
        
        return self._output

    # Calculate Error Node Output
    def _calculate_error_node_ouput(self, target, isOutput):
        print(f"[●] Calculating Error Node Output for layer {self._id}")

        # For each output, calculate its error node output
        for i in range(self._num_of_neuron):
            self._calculate_error(target, i, isOutput)
            self._calculate_derivative(i)

        print(f"[✔] Successfully calculated Error Node Output for layer {self._id}")
        for i in range(len(self._error_node_output)):
            print(f"e{self._id}{i}: {self._error_node_output[i]}")
        print("")

        return self._error_node_output

    def _calculate_loss(self, target):
        print(f"[●] Calculating Loss for layer {self._id}")

        for i in range(self._num_of_neuron):
            self._loss += (target[i] - self._output[i]) ** 2

        self._loss /= len(target)

        print(f"[✔] Successfully calculated Loss for layer {self._id}")
        print(f"Loss: {self._loss}")
        print("")

        return self._loss
        

In [274]:
class FFNN:
    # Initialize model
    def __init__(self, model):
        try:
            self._layers = model["case"]["model"]["layers"]
            self._weights = model["case"]["initial_weights"]
            self._inputs = model["case"]["input"]
            self._targets = model["case"]["target"]
            self._layer_objects = []

            self._learning_rate = model["case"]["learning_parameters"]["learning_rate"]
            self._batch_size = model["case"]["learning_parameters"]["batch_size"]
            self._max_iteration = model["case"]["learning_parameters"]["max_iteration"]
            self._error_threshold = model["case"]["learning_parameters"]["error_threshold"]

            self._error_node_output = []
            self._loss = 0
            
            self._outputs = []
            self._outputs
            self._expect = model["case"]["expect"]
        except:
            print(f"[✖] Error initialiazing model")
            print("")

            Driver.terminate(1)
        
        print(f"[✔] Successfully initialized model")
        print(f"Number of features: {model['case']['model']['input_size']}")
        print(f"Number of layers: {len(self._layers)}")
        print(f"Number of data: {len(self._inputs)}")
        print("")

    # Assert the output with the expected output
    def test(self):
        print(f"[●] Asserting output =======================================")

        passed_output = 0
        for i in range(len(self._inputs)):
            if (np.all(self._expect['output'][i] < self._outputs[i] + self._expect['max_sse']) or np.all(self._expect['output'][i] > self._outputs[i] - self._expect['max_sse'])):
                passed_output += 1

        print(f"Test status: {'FAIL' if passed_output != len(self._inputs) else 'PASS'}")
        print(f"Test result: {passed_output}/{len(self._inputs)}")
        print("")
    
    # Build the network
    def build(self):
        # Loop until max epoch
        for epoch in range(self._max_iteration):
            print(f"[●] Epoch {epoch + 1} ========================================")

            # Split data into batches
            for batch_start in range(0, len(self._inputs), self._batch_size):
                X_batch = self._inputs[batch_start:batch_start + self._batch_size]
                y_batch = self._targets[batch_start:batch_start + self._batch_size]

                self._outputs = []
                self._error_node_output = []
                self._layer_objects = []

                print(f"[●] Building batch {batch_start + 1}")

                # For each data in batch, calculate output
                for i in range(len(X_batch)):
                    _in = X_batch[i]
                    layers = []

                    # Feed forward
                    for j in range(len(self._layers)):
                        
                        # Create layer
                        layer = Layer(j, self._layers[j], self._weights[j], _in)

                        # Calculate forward
                        _in = layer._calculate_forward()

                        # Calculate Loss
                        loss = layer._calculate_loss(y_batch[i])

                        layers.append(layer)           
                        self._outputs.append(_in)
                    
                    self._layer_objects.append(layers)
                    self._loss += loss
                    print(f"[✔] Successfully evaluated data {i + 1}")

                # Calculate Error Node Output
                for i in range(len(X_batch)):
                    for j in range(len(self._layers) - 1, -1, -1):
                        print(f"[●] Calculating Error Node Output for data {i + 1} in layer {j}")
                        print(self._layer_objects[i][j]._output)
                        if (j == len(self._layers) - 1):
                            self._error_node_output.append(self._layer_objects[i][j]._calculate_error_node_ouput(y_batch[i], True))
                        else:
                            self._error_node_output.append(self._layer_objects[i][j]._calculate_error_node_ouput(self._layer_objects[i][j + 1]._error_node_output, False))

                # Update weights
                m = len(self._layers) - 1
                for i in range(len(X_batch)):
                    for j in range(len(self._layers)):
                        for k in range(len(self._weights[j])):
                            for l in range(len(self._weights[j][k])):
                                if k == 0:
                                    self._weights[j][k][l] += self._learning_rate * self._error_node_output[m][l]
                                else:
                                    self._weights[j][k][l] += self._learning_rate * self._error_node_output[m][l] * X_batch[i][k - 1]
                        m -= 1

                # Loss threshold
                self._loss /= len(X_batch)
                if self._loss <= self._error_threshold:
                    print(f"[✔] Loss threshold reached")
                    return

                # print weights
                print(self._weights)

In [275]:
model_file = 'models/relu.json'

In [276]:
# Input
model_json = Driver.load_model(model_file)

# Initialize FFNN
model = FFNN(model_json)

# Build FNN
model.build()

# Test the output
# model.test()


[✔] Successfully loaded model from 'models/relu.json'

[✖] Error initialiazing model

[■] Program terminated with code 1
[✔] Successfully initialized model
Number of features: 2
Number of layers: 1
Number of data: 2

[●] Epoch 1 ========================================
[●] Building batch 1
[●] Building layer 0
[✔] Successfully built layer 0
Number of neurons: 3
Activation function: relu
h00: 0.04999999999999993
h01: 1.1
h02: 0.0

[●] Calculating Loss for layer 0
[✔] Successfully calculated Loss for layer 0
Loss: 0.007500000000000009

[✔] Successfully evaluated data 1
[●] Building layer 0
[✔] Successfully built layer 0
Number of neurons: 3
Activation function: relu
h00: 0.0
h01: 0.0
h02: 1.5

[●] Calculating Loss for layer 0
[✔] Successfully calculated Loss for layer 0
Loss: 0.09000000000000001

[✔] Successfully evaluated data 2
[●] Calculating Error Node Output for data 1 in layer 0
[0.05 1.1  0.  ]
[●] Calculating Error Node Output for layer 0
[✔] Successfully calculated Error Node Ou

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ad693be4-3a1e-4be5-9f4f-9f4a320647d1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>